In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%pip install transformers
%pip install torch
%pip install pandas
%pip install datasets

In [3]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset, random_split
from transformers import BertTokenizer, BertForSequenceClassification, RobertaTokenizer, RobertaForSequenceClassification
from transformers import AdamW, get_linear_schedule_with_warmup
import torch.nn.functional as F
import numpy as np
from torch.nn.utils.rnn import pad_sequence
import torch.optim as optim
from sklearn.model_selection import train_test_split
import torch.nn as nn
from sklearn.metrics import accuracy_score
from tqdm import tqdm

In [4]:
# Load the dataset
file_path = '/content/drive/MyDrive/4+1/NLP/Final/cancer_doc_dataset.csv'
df = pd.read_csv(file_path)

In [5]:
# Map labels to integers
label_map = {label: idx for idx, label in enumerate(df['Label'].unique())}
df['Label'] = df['Label'].map(label_map)

In [6]:
class CancerDocDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts.reset_index(drop=True)
        self.labels = labels.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

In [7]:
# Split the dataset into train, validation, and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['Text'], df['Label'], test_size=0.2, random_state=42, stratify=df['Label']
)

val_texts, test_texts, val_labels, test_labels = train_test_split(
    test_texts, test_labels, test_size=0.5, random_state=42, stratify=test_labels
)


In [8]:
# DataLoader
def create_data_loader(texts, labels, tokenizer, max_len, batch_size):
    ds = CancerDocDataset(texts, labels, tokenizer, max_len)
    return DataLoader(ds, batch_size=batch_size, num_workers=4)

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
# Model training function
def train_epoch(model, data_loader, loss_fn, optimizer, device, scheduler, n_examples):
    model = model.train()
    losses = 0
    correct_predictions = 0

    for data in tqdm(data_loader, desc="Training"):
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        labels = data['label'].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits
        _, preds = torch.max(logits, dim=1)

        correct_predictions += torch.sum(preds == labels)
        losses += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()

    return correct_predictions.double() / n_examples, losses / n_examples

In [11]:
# Model evaluation function
def eval_model(model, data_loader, loss_fn, device, n_examples):
    model = model.eval()
    losses = 0
    correct_predictions = 0

    with torch.no_grad():
        for data in tqdm(data_loader, desc="Evaluating"):
            input_ids = data['input_ids'].to(device)
            attention_mask = data['attention_mask'].to(device)
            labels = data['label'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits
            _, preds = torch.max(logits, dim=1)

            correct_predictions += torch.sum(preds == labels)
            losses += loss.item()

    return correct_predictions.double() / n_examples, losses / n_examples

In [12]:
# Prediction function
def predict(model, data_loader, device):
    model = model.eval()
    texts = []
    predictions = []
    real_values = []

    with torch.no_grad():
        for data in tqdm(data_loader, desc="Predicting"):
            input_ids = data['input_ids'].to(device)
            attention_mask = data['attention_mask'].to(device)
            labels = data['label'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            _, preds = torch.max(logits, dim=1)

            texts.extend(data['input_ids'].cpu().numpy())
            predictions.extend(preds.cpu().numpy())
            real_values.extend(labels.cpu().numpy())

    for text, pred, real in zip(texts, predictions, real_values):
        print(f'Text: {text}')
        print(f'Prediction: {pred}, Real: {real}')
        print()

In [13]:
# Parameters
EPOCHS = 4 #Used to be 10 but the performance was very good
EMBED_DIM = 512  # Adjust according to your model configuration
SRC_VOCAB_SIZE = 30522  # BERT tokenizer vocabulary size
TARGET_VOCAB_SIZE = 2
BATCH_SIZE = 32
MAX_LEN = 512

# **BERT**

In [ ]:
# Preprocess the dataset
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
bert_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=df['Label'].nunique())
bert_model = bert_model.to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
bert_train_data_loader = create_data_loader(train_texts, train_labels, bert_tokenizer, MAX_LEN, BATCH_SIZE)
bert_val_data_loader = create_data_loader(val_texts, val_labels, bert_tokenizer, MAX_LEN, BATCH_SIZE)
bert_test_loader = create_data_loader(test_texts, test_labels, bert_tokenizer, MAX_LEN, BATCH_SIZE)

In [ ]:
optimizer = AdamW(bert_model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(bert_train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
loss_fn = torch.nn.CrossEntropyLoss().to(device)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
print("Training BERT model...")
for epoch in range(EPOCHS):
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    print('-' * 10)
    train_acc, train_loss = train_epoch(bert_model, bert_train_data_loader, loss_fn, optimizer, device, scheduler, len(train_texts))
    print(f'Train loss {train_loss} accuracy {train_acc}')
    val_acc, val_loss = eval_model(bert_model, bert_val_data_loader, loss_fn, device, len(val_texts))
    print(f'Val loss {val_loss} accuracy {val_acc}')
    print()

Training BERT model...
Epoch 1/4
----------


Training: 100%|██████████| 190/190 [03:58<00:00,  1.25s/it]


Train loss 0.011694598628350754 accuracy 0.8180317040951123


Evaluating: 100%|██████████| 24/24 [00:19<00:00,  1.24it/s]


Val loss 0.0031929120219643553 accuracy 0.9749009247027741

Epoch 2/4
----------


...

Val loss 0.0010139783920859857 accuracy 0.9920739762219287

Epoch 4/4
----------


Training: 100%|██████████| 190/190 [03:59<00:00,  1.26s/it]


Train loss 0.0005801844640333997 accuracy 0.9927344782034346


Evaluating: 100%|██████████| 24/24 [00:19<00:00,  1.24it/s]

Val loss 0.0005873306297612237 accuracy 0.9933949801849405



In [ ]:
predict(bert_model, bert_test_loader, device)

Predicting: 100%|██████████| 24/24 [00:19<00:00,  1.24it/s]


Streaming output truncated to the last 5000 lines.
 21181  1997 19314 17919  2118  5548 25196  2710  3993  2140  2862  1997
  3166  2592  2003  2800  2012  1996  2203  1997  1996  5493  2011  1038
  1062  4048 22994 21983 18447 14127 14194   102]
Prediction: 1, Real: 1

Text: [  101  1000  4319  5012  5260  2000 13656  2128  2721 29251  1998  2582
 14967  2076 27144  1999 11192  4456  2485 24004 21197  1997  1048  2487
 10381  2140  2487  2038  2042  4453  2004  1037 13656 16081  2953  1999
  2087 16007 28207  9243  2174  2000  1996  2190  1997  2256  3716  3251
 10381  2140  2487  2865  4570 18178  5974  6190 26897  3464  4242  1996
  2556  2817  5159  2008 10381  2140  2487  2001  6022  2091  2890 24848
  4383  1999 20199 24759 20363 20315 28994 23137  2102  4442  1037 27009
  2683 14141  2361  1998 14397 27606  2595  2884 13866  2595  6072 23137
  2102  4442  1037 27009  2683 13876  2595  4102  2007  1037 27009  2683
  4442  2043  5845  2007  2030  2302 20315  2361  1998 13866  2595

# **ROBERTA**

In [14]:
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta_model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=df['Label'].nunique())
roberta_model = roberta_model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
roberta_train_data_loader = create_data_loader(train_texts, train_labels, roberta_tokenizer, MAX_LEN, BATCH_SIZE)
roberta_val_data_loader = create_data_loader(val_texts, val_labels, roberta_tokenizer, MAX_LEN, BATCH_SIZE)
roberta_test_loader = create_data_loader(test_texts, test_labels, roberta_tokenizer, MAX_LEN, BATCH_SIZE)

In [16]:
optimizer = AdamW(roberta_model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(roberta_train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
loss_fn = torch.nn.CrossEntropyLoss().to(device)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [17]:
print("Training RoBERTa model...")
for epoch in range(EPOCHS):
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    print('-' * 10)
    train_acc, train_loss = train_epoch(roberta_model, roberta_train_data_loader, loss_fn, optimizer, device, scheduler, len(train_texts))
    print(f'Train loss {train_loss} accuracy {train_acc}')
    val_acc, val_loss = eval_model(roberta_model, roberta_val_data_loader, loss_fn, device, len(val_texts))
    print(f'Val loss {val_loss} accuracy {val_acc}')
    print()

Training RoBERTa model...
Epoch 1/4
----------


Training: 100%|██████████| 190/190 [05:05<00:00,  1.61s/it]


Train loss 0.0018466117228060116 accuracy 0.9753963011889035


Evaluating: 100%|██████████| 24/24 [00:15<00:00,  1.56it/s]


Val loss 3.2634331163171497e-06 accuracy 1.0

Epoch 2/4
----------


...

Val loss 1.954544900802408e-06 accuracy 1.0

Epoch 4/4
----------


Training: 100%|██████████| 190/190 [05:04<00:00,  1.60s/it]


Train loss 2.789211031714226e-06 accuracy 1.0


Evaluating: 100%|██████████| 24/24 [00:15<00:00,  1.55it/s]

Val loss 1.8357370047838657e-06 accuracy 1.0



In [18]:
predict(roberta_model, roberta_test_loader, device)

Predicting: 100%|██████████| 24/24 [00:15<00:00,  1.51it/s]


Streaming output truncated to the last 5000 lines.
  3551 30708  4982  1368  6852  7309  3707  9041    29    55    87  1437
  1484   228    76  1092 11942  3290 15758 28033  7443  5776 36123  2196
  4971     5  1374  1416     5  7601   490     2]
Prediction: 1, Real: 1

Text: [    0   113 41533  5910  3315     7 16570 38387     8   617 16739  1437
   148  1437 17930  1437    11  1437 10665  1437  1668  1437 10041  9486
  8982     9   226   134  3858   574   134    34    57  2006    25    10
 16570 23192   368    11   144  8196  4932 27661   635     7     5   275
     9    84  2655   549  3858   574   134 18422  1626 28033  4765 32556
  1189  4727    20  1455   892  6373    14  3858   574   134    21  3625
   159 35908    11 39307  2911 16779   211  5174 21808  4590    83 36476
   495  5174     8 18234 13534  3631   523  7008  1000 21808  4590    83
 36476 10311  1000  1118    19    83 36476  4590   520  3032    19    50
   396   211  5174     8  7008  1000 11052 17725     9  3858   574

Việc sử dụng mô hình đã được tiền huấn luyện (**pretrained model**) thường mang lại kết quả tốt hơn so với việc huấn luyện một mô hình từ đầu (**training from scratch**) với nhiều lý do:

1. **Kiến thức được tích lũy trước:** Mô hình tiền huấn luyện được huấn luyện trên một lượng lớn dữ liệu trước đó, cho phép chúng học được một bộ biểu diễn phong phú và đa dạng về ngôn ngữ, hình ảnh, hoặc các loại dữ liệu khác. Khi bạn sử dụng một mô hình như vậy cho tác vụ cụ thể của mình, mô hình đã có sẵn kiến thức cơ bản về cách xử lý dữ liệu và chỉ cần điều chỉnh nhỏ để tối ưu cho tác vụ đặc biệt đó.

2. **Yêu cầu dữ liệu ít hơn:** Do đã được tiền huấn luyện trên một tập dữ liệu lớn, mô hình có khả năng tổng quát hóa tốt hơn từ một lượng dữ liệu nhỏ hơn trong quá trình tinh chỉnh (fine-tuning) cho một tác vụ cụ thể. Điều này đặc biệt quan trọng khi bạn không có quyền truy cập vào lượng lớn dữ liệu được gán nhãn cho tác vụ của mình.

3. **Giảm thời gian và chi phí huấn luyện:** Huấn luyện một mô hình lớn từ đầu đòi hỏi rất nhiều tài nguyên tính toán và thời gian. Sử dụng mô hình tiền huấn luyện cho phép bạn tận dụng kiến thức mà mô hình đã học được, giảm đáng kể thời gian và chi phí cần thiết để đạt được hiệu suất tốt.

4. **Tăng cường hiệu suất:** Mô hình tiền huấn luyện thường đạt được hiệu suất tốt trên nhiều tác vụ khác nhau, bởi vì chúng đã học được các đặc trưng tổng quát từ dữ liệu lớn. Khi tinh chỉnh mô hình cho một tác vụ cụ thể, bạn thường có thể đạt được hiệu suất tốt hơn so với việc huấn luyện từ đầu, đặc biệt là khi tác vụ của bạn tương tự với tác vụ mà mô hình đã được tiền huấn luyện.

5. **Khả năng chuyển giao kiến thức:** Các mô hình tiền huấn luyện có khả năng chuyển giao kiến thức từ tác vụ này sang tác vụ khác. Điều này có nghĩa là kiến thức được mô hình học từ tác vụ A có thể hữu ích khi giải quyết tác vụ B, qua đó cải thiện hiệu suất trên tác vụ B mà không cần một lượng lớn dữ liệu được gán nhãn cụ thể cho tác vụ đó.